In [1]:
import pandas as pd
import matplotlib.pyplot as plt

Se han calculado los salarios medios netos a partir de los datos obtenidos en https://gestiona.comunidad.madrid/desvan/AccionListadoTematicoDesvan.icm?temaPadre=432&esTema=S&descripcionPadre=Salario%20medio%20anual%20por%20grupos%20de%20edad.%20Espa%F1oles&codTema=1902636. Los salarios utilizados son los de 2022, debido a que se encuentran a medio camino a nivel de inflación entre los datos de coste de vida medios para Madrid (2024) y el dataset de precios de vivienda en Madrid (2020) buscando compensar la falta de sincronización, que puede afectar ligeramente a los resultados si no se tiene en cuenta.

Se ha usado la calculadora de IRPF del diario Cinco Días, de El País. Es la calculadora que llevo usando varios años para calcular mi cuota de IRPF y funciona de manera precisa.

Se han calculado los salarios y se han definido como constantes.

In [2]:
MONTHLY_SALARY_AT_26_35 = 1680
MONTHLY_SALARY_AT_36_45 = 2162
MONTHLY_SALARY_AT_46_55 = 2472
MONTHLY_SALARY_AT_56_65 = 2329

In [3]:
MEAN_COST_OF_RENT_1_BEDROOM = 1162.5
MEAN_COST_OF_RENT_3_BEDROOM = 1905.5


Se definen los grupos a analizar:
- El grupo 1, es decir, personas hasta 36 años, no tienen ahorros y todavía no tienen hijos. Por facilitar la historia, en este grupo todos están alquilados.
- Para el grupo 2 asumimos una familia que compra un piso, con 2 hijos de media y con cierto nivel de ahorros, teniendo en cuenta que tienen hijos en torno a los 35 años y han podido ahorrar durante 10 años (desde los 25).
- Para el grupo 3 asumimos una familia de 2 hijos que ya tiene una vivienda que paga mediante una hipoteca y quiere adquirir otra. Es difícil acceder a los datos de hipotecas pasadas, pero lo normal hasta ahora es que las letras fuesen inferiores al alquiler medio, por lo que vamos a establecer una cuota de 1100 euros como gasto extra para este grupo. Se ha fijado una cuota por debajo del alquiler pensando en pisos que se compraron antes de la crisis del 2008, pero no necesariamente dentro de los años de la carrera alcista provocada por la burbuja inmobiliaria. El metro cuadrado se pagaba a principios de 2006 a unos 3.500 € de media (fuente: Idealista), por lo que un piso de 90 metros cuadrados con 3 habitaciones podría costar en torno a unos 315.000 €. Asumiendo unos 100.000 € de entrada, con unos tipos que oscilaban entre el 4 y el 5%, es decir, promediando al 4.5%, se queda una hipoteca de 1200 € aproximadamente. No obstante, como los precios sufrieron una escalada entre 2005 y 2008, se descuenta un pequeño porcentaje para compensar algunas hipotecas firmadas antes de ese periodo a un precio más razonable.
- Para el grupo 4 nos basamos en familias que ya tienen hijos mayores de 25 años, posiblemente independizados, y que ya han pagado su casa. Los costes, por tanto, serán para 2 personas, como para el grupo 1, pero sin el gasto del alquiler o la hipoteca que tienen el resto de grupos.

In [4]:
PENDING_MORTGAGE_COST = 1100
MEAN_COST_WITHOUT_RENT_INDIVIDUAL = 788
MEAN_COST_WITHOUT_RENT_PER_FAMILY = 2768

Por tanto, generamos los costes mensuales por grupo

In [5]:
GROUP_26_35_TOTAL_COSTS = MEAN_COST_OF_RENT_1_BEDROOM + 2*MEAN_COST_WITHOUT_RENT_INDIVIDUAL
GROUP_36_45_TOTAL_COSTS = MEAN_COST_OF_RENT_1_BEDROOM + MEAN_COST_WITHOUT_RENT_PER_FAMILY
GROUP_46_55_TOTAL_COSTS = PENDING_MORTGAGE_COST + MEAN_COST_WITHOUT_RENT_PER_FAMILY
GROUP_56_65_TOTAL_COSTS = 2*MEAN_COST_WITHOUT_RENT_INDIVIDUAL

In [6]:
print(f'Cost of living for people between 26 and 35: {GROUP_26_35_TOTAL_COSTS}')
print(f'Cost of living for people between 36 and 45: {GROUP_36_45_TOTAL_COSTS}')
print(f'Cost of living for people between 46 and 55: {GROUP_46_55_TOTAL_COSTS}')
print(f'Cost of living for people between 56 and 65: {GROUP_56_65_TOTAL_COSTS}')

Cost of living for people between 26 and 35: 2738.5
Cost of living for people between 36 and 45: 3930.5
Cost of living for people between 46 and 55: 3868
Cost of living for people between 56 and 65: 1576


In [7]:
GROUP_26_35_MONTHLY_SAVINGS = 2*MONTHLY_SALARY_AT_26_35 - GROUP_26_35_TOTAL_COSTS
GROUP_36_45_MONTHLY_SAVINGS = 2*MONTHLY_SALARY_AT_36_45 - GROUP_36_45_TOTAL_COSTS
GROUP_46_55_MONTHLY_SAVINGS = 2*MONTHLY_SALARY_AT_46_55 - GROUP_46_55_TOTAL_COSTS
GROUP_56_65_MONTHLY_SAVINGS = 2*MONTHLY_SALARY_AT_56_65 - GROUP_56_65_TOTAL_COSTS

In [8]:
print(f'Monthly savings for people between 26 and 35: {GROUP_26_35_MONTHLY_SAVINGS}')
print(f'Monthly savings for people between 36 and 45: {GROUP_36_45_MONTHLY_SAVINGS}')
print(f'Monthly savings for people between 46 and 55: {GROUP_46_55_MONTHLY_SAVINGS}')
print(f'Monthly savings for people between 56 and 65: {GROUP_56_65_MONTHLY_SAVINGS}')

Monthly savings for people between 26 and 35: 621.5
Monthly savings for people between 36 and 45: 393.5
Monthly savings for people between 46 and 55: 1076
Monthly savings for people between 56 and 65: 3082


Se generan los esfuerzos para cada grupo

In [22]:
data = pd.read_csv('/Users/luisgarciafresno/pra_ii/datasets/analyzed_data/clean_after_null.csv')
data["years_of_saving_for_26_35"] = (data["buy_price"]*0.3 + 0.06*data['buy_price'] + 2700) / (GROUP_26_35_MONTHLY_SAVINGS*12)
data["years_of_saving_for_36_45"] = (data["buy_price"]*0.3 + 0.06*data['buy_price'] + 2700) / (GROUP_36_45_MONTHLY_SAVINGS*12)
data["years_of_saving_for_46_55"] = (data["buy_price"]*0.3 + 0.06*data['buy_price'] + 2700) / (GROUP_46_55_MONTHLY_SAVINGS*12)
data["years_of_saving_for_56_65"] = (data["buy_price"]*0.3 + 0.06*data['buy_price'] + 2700) / (GROUP_56_65_MONTHLY_SAVINGS*12)


In [23]:
data[['buy_price', 'years_of_saving_for_26_35', 'years_of_saving_for_36_45', 'years_of_saving_for_46_55', 'years_of_saving_for_56_65']]

,buy_price,years_of_saving_for_26_35,years_of_saving_for_36_45,years_of_saving_for_46_55,years_of_saving_for_56_65
0,85000,4.465004,7.052097,2.578996,0.900389
1,129900,6.632341,10.475222,3.830855,1.337443
2,144247,7.324875,11.569022,4.230864,1.477096
3,109900,5.666935,8.950445,3.273234,1.142764
4,260000,12.912309,20.393901,7.458178,2.603829
...,...,...,...,...,...
18086,360000,17.739340,28.017789,10.246283,3.577223
18087,349000,17.208367,27.179161,9.939591,3.470149
18088,350000,17.256637,27.255400,9.967472,3.479883
18089,425000,20.876911,32.973316,12.058550,4.209929


In [25]:
data.drop(
    'Unnamed: 0',
    axis=1,
    inplace=True
)

Se analizan y eliminan los outliers de la muestra de datos

In [30]:
# Step 1: Calculate Q1, Q3, and IQR
Q1 = data["buy_price"].quantile(0.25)
Q3 = data["buy_price"].quantile(0.75)
IQR = Q3 - Q1

# Step 2: Define outlier boundaries
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Step 3: Identify outliers
outliers = data[(data["buy_price"] < lower_bound) | (data["buy_price"] > upper_bound)]

# Step 4: Print outlier summary
print(f"Outlier analysis for 'buy_price':")
print(f"Lower bound: {lower_bound}")
print(f"Upper bound: {upper_bound}")
print(f"Number of outliers: {len(outliers)}")

# Remove outliers
data_no_outliers = data[(data["buy_price"] >= lower_bound) & (data["buy_price"] <= upper_bound)]

Outlier analysis for 'buy_price':
Lower bound: -502500.0
Upper bound: 1357500.0
Number of outliers: 1537


In [31]:
data_no_outliers.to_csv('/Users/luisgarciafresno/pra_ii/datasets/analyzed_data/final_data_no_outliers.csv')

In [36]:
# Filter 1: Conditions for sq_mt_built > 60, n_rooms >= 1, has_lift = 1, is_exterior = 1
condition1 = (
    (data_no_outliers['n_rooms'] >= 1) &
    (data_no_outliers['has_lift'] == 1) &
    (data_no_outliers['is_exterior'] == 1) &
    (data_no_outliers['sq_mt_built'] > 60)
)
filtered1 = data_no_outliers[condition1]

# Calculate medians for Filter 1
medians1 = {
    "buy_price_median": filtered1['buy_price'].median(),
    "years_of_saving_for_26_35_median": filtered1['years_of_saving_for_26_35'].median(),
    "years_of_saving_for_36_45_median": filtered1['years_of_saving_for_36_45'].median(),
    "years_of_saving_for_46_55_median": filtered1['years_of_saving_for_46_55'].median(),
    "years_of_saving_for_56_65_median": filtered1['years_of_saving_for_56_65'].median(),
}

# Filter 2: Conditions for sq_mt_built > 89, n_rooms >= 2, has_lift = 1, is_exterior = 1
condition2 = (
    (data_no_outliers['n_rooms'] >= 2) &
    (data_no_outliers['has_lift'] == 1) &
    (data_no_outliers['is_exterior'] == 1) &
    (data_no_outliers['sq_mt_built'] > 89)
)
filtered2 = data_no_outliers[condition2]

# Calculate medians for Filter 2
medians2 = {
    "buy_price_median": filtered2['buy_price'].median(),
    "years_of_saving_for_26_35_median": filtered2['years_of_saving_for_26_35'].median(),
    "years_of_saving_for_36_45_median": filtered2['years_of_saving_for_36_45'].median(),
    "years_of_saving_for_46_55_median": filtered2['years_of_saving_for_46_55'].median(),
    "years_of_saving_for_56_65_median": filtered2['years_of_saving_for_56_65'].median(),
}

# Create a new dataset with the results
results = pd.DataFrame([
    {"filter": "sq_mt_built > 60, n_rooms >= 1", **medians1},
    {"filter": "sq_mt_built > 89, n_rooms >= 2", **medians2},
])

# Save the results
output_path = '/Users/luisgarciafresno/pra_ii/datasets/analyzed_data/filtered_median_data.csv'
results.to_csv(output_path, index=False)

In [37]:
results

,filter,buy_price_median,years_of_saving_for_26_35_median,years_of_saving_for_36_45_median,years_of_saving_for_46_55_median,years_of_saving_for_56_65_median
0,"sq_mt_built > 60, n_rooms >= 1",463000.0,22.711183,35.870394,13.118030,4.579818
1,"sq_mt_built > 89, n_rooms >= 2",564300.0,27.600965,43.593393,15.942379,5.565866
